转自博文：https://blog.csdn.net/u011808673/article/details/80609221  
程序使用一段时间后会遇到HTTP Error 403: Forbidden错误。 因为在短时间内直接使用Get获取大量数据，会被服务器认为在对它进行攻击，所以拒绝我们的请求，自动把电脑IP封了。

In [11]:
import requests
import random

### 1.增加header

In [12]:

# 收集到的常用Header
header_choices = [
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "]

In [13]:
headvalue=dict()
headvalue['User-Agent']=random.choice(header_choices)
url='https://www.baidu.com'
r=requests.get(url,headers=headvalue)
r.raise_for_status()
r.encoding=r.apparent_encoding

但有时这样的做法也不一定有用。前面也说到，服务器是根据IP判断。 给请求增加Header只是伪装成不同的浏览器而已。如果同一个IP在短时间内频繁访问， 就算是浏览器请求也会被拒绝掉。因此对于这个问题就只好从另一个方面着手，即适当降低单个IP访问频率。 对于每个IP而言，每次请求操作之间都暂停一段时间。同时利用多个IP进行访问。通过这两种手段可以降低被拒绝的可能性。


### 2. 代理IP

简单地说是通过自动更换不同IP来“迷惑”服务器，让它认为是来自不同电脑的访问请求， 从而不会被拒绝掉。由于代理IP的时效性很强，所以需要经常更换。最好是“现用现找”。

In [17]:
# 设置代理IP地址（字典形式）
proxy={
"http":"http://211.149.172.228:9999",
"https":"https://116.196.90.181:3128",
}

url="http://www.sogou.com"

# 创建请求，传入代理参数；获取响应数据
resp=requests.get(url,proxies=proxy)
data=resp.text  # 以文本形式获取响应信息

ProxyError: HTTPConnectionPool(host='211.149.172.228', port=9999): Max retries exceeded with url: http://www.sogou.com/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108eb7940>: Failed to establish a new connection: [Errno 60] Operation timed out')))

In [ ]:
print(resp)
print(data)

### 3.组合办法

说了上面两种方法后，很自然地就会想到把两种方法结合起来。这样就会大大提高请求的种类。 如在下面的代码中Header有13个，IP有6个，排列组合就有78中请求。从理论上来说， 组合数越多就越不容易被封。同时再加上请求延迟，是较好的解决方案。